In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import norm ,skew

In [ ]:
df_train=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_train.csv')

In [ ]:
df_train.info(memory_usage='deep')

In [ ]:
c_int=df_train.select_dtypes(include=['int64']).apply(pd.to_numeric,downcast='unsigned')
c_float=df_train.select_dtypes(include=['float']).apply(pd.to_numeric,downcast='float')


In [ ]:
o_df=pd.concat([c_int,c_float],axis=1)

In [ ]:
column_type=dict(o_df.dtypes.copy())

In [ ]:
len(column_type)

In [ ]:
del o_df,c_int,c_float,df_train

In [ ]:
data_train=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_train.csv',
                      dtype=column_type,parse_dates=['register_time'],infer_datetime_format=True)

In [ ]:
data_train.rename(columns={'treatment_acceleraion_add_value':'treatment_acceleration_add_value',\
                          'sr_rss_a_prod_levell':'sr_rss_a_prod_level'} ,inplace=True)

In [ ]:
data_train.head()

In [ ]:
data_train.info(memory_usage='deep')

In [ ]:
# data_label=data_train['prediction_pay_price']
data_train2=data_train.drop(labels=['user_id','register_time'],axis=1)

In [ ]:
data_7pay=data_train2[data_train2['pay_price']>0].copy()


In [ ]:
data_7pay.head()

In [ ]:
#######################################

In [ ]:
# plt.scatter(data_7pay.pay_price,data_7pay.prediction_pay_price)
# plt.show()

In [ ]:
# plt.subplot(1,2,1)
# sns.distplot(data_7pay.prediction_pay_price,kde=False,fit=norm)

# plt.subplot(122)
# sns.distplot(np.log(data_7pay.prediction_pay_price),kde=False,fit=norm)
# plt.show()

In [ ]:
skew(data_7pay.prediction_pay_price)

In [ ]:
skew(np.log(data_7pay.prediction_pay_price))

In [ ]:
data_7pay['prediction_pay_price']=np.log(data_7pay.prediction_pay_price)

In [ ]:
numeric_dtypes=['uint16','uint32','uint64','float16','float32','float64']
numeric2=[]
for i in data_7pay.columns:
    if data_7pay[i].dtype in numeric_dtypes:
        numeric2.append(i)
skew_data_7pay=data_7pay[numeric2].apply(lambda x:skew(x)).sort_values(ascending=False)
skews=pd.DataFrame({'skew':skew_data_7pay})
skews

In [ ]:
data_7pay.info(memory_usage='deep')

In [ ]:
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

In [ ]:
high_skew=skew_data_7pay[skew_data_7pay>0.5]
high_skew=high_skew
skew_index=high_skew.index

for i in skew_index:
    data_7pay[i]=boxcox1p(data_7pay[i],boxcox_normmax(data_7pay[i]+1))
skew_data_7pay2=data_7pay[numeric2].apply(lambda x:skew(x)).sort_values(ascending=False)
skew2=pd.DataFrame({'skew':skew_data_7pay2})
skew2

In [ ]:
data_7pay.shape

In [56]:
drop_col=[i for i in data_7pay.columns.values if 'prediction' in i]
X=data_7pay.drop(labels=drop_col,axis=1)
y=data_7pay.prediction_pay_price
X.replace([np.inf,-np.inf],np.nan,inplace=True)
X.fillna(0.0,inplace=True)
from sklearn.model_selection import cross_val_score,KFold
from sklearn.linear_model import LinearRegression,RidgeCV,LassoCV,ElasticNetCV
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline

kfolds=KFold(n_splits=10,shuffle=True,random_state=13)


def cv_rmse(model):
    rmse=np.sqrt(-cross_val_score(model,X,y,scoring='neg_mean_squared_error',cv=kfolds))
    return rmse

lm = LinearRegression()

benchmark_model=make_pipeline(RobustScaler(),
                             lm).fit(X,y)

cv_rmse(benchmark_model).mean()

0.38540743873458705

In [57]:
np.expm1(0.385)

0.4696143214411443

In [58]:
r_alpha=np.arange(10,21,1)
def ridge_selector(k):
    ridge_model=make_pipeline(RobustScaler(),
                             RidgeCV(alphas=[k],cv=kfolds)).fit(X,y)
    return cv_rmse(ridge_model).mean()

r_score=[]
for i in r_alpha:
    score=ridge_selector(i)
    r_score.append(score)    

In [59]:
r_score_df=pd.DataFrame(list(zip(r_alpha,r_score)),columns=['Alpha','RMSE'])

In [60]:
r_score_df.sort_values(by='RMSE')

,Alpha,RMSE
5,15,0.385371
6,16,0.385371
4,14,0.385371
7,17,0.385371
3,13,0.385371
8,18,0.385371
2,12,0.385372
9,19,0.385372
1,11,0.385372
10,20,0.385372


In [61]:
r_alpha2=pd.Series(np.arange(14,16,0.1)).apply(lambda x:round(x,2))

ridge_model=make_pipeline(RobustScaler(),
                         RidgeCV(alphas=r_alpha2.tolist(),cv=kfolds)).fit(X,y)
cv_rmse(ridge_model).mean()

0.3853710941224412

In [62]:
ridge_model.steps[1][1].alpha_

15.9

In [ ]:
# l_alpha=[0.01,0.1,0.5,1,5,10,15,20,30,50]
# l_alpha2=pd.Series(np.arange(0.001,0.02,0.002)).apply(lambda x:round(x,3))
l_alpha2 = [0.00001,0.00002,0.00003,0.00004,0.00005, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005,
           0.0006, 0.0007, 0.0008]

lasso_model=make_pipeline(RobustScaler(),
                         LassoCV(alphas=l_alpha2,max_iter=1e5,cv=kfolds)).fit(X,y)
cv_rmse(lasso_model).mean()

In [ ]:
lasso_model.steps[1][1].alpha_

In [ ]:
la_coef=pd.DataFrame(list(zip(X.columns,lasso_model.steps[1][1].coef_)),columns=['predictors','coef'])
la_coef.sort_values(by='coef',ascending=False)

In [ ]:
la_coef[la_coef.coef == 0].shape

In [ ]:
used_la_coef=la_coef[la_coef.coef !=0]['predictors']
used_la_coef.shape

In [ ]:
X[used_la_coef].shape

In [ ]:
X2=X[used_la_coef]

In [ ]:
def cv_rmse2(model):
    rmse=np.sqrt(-cross_val_score(model,X2,y,scoring='neg_mean_squared_error',cv=kfolds))
    return rmse

In [ ]:
r_alpha2=pd.Series(np.arange(10,12,0.1)).apply(lambda x:round(x,1))
# r_alpha=np.arange(1,15,1)
ridge_model=make_pipeline(RobustScaler(),
                         RidgeCV(alphas=r_alpha2.tolist(),cv=kfolds)).fit(X2,y)
cv_rmse2(ridge_model).mean()

In [ ]:
ridge_model.steps[1][1].alpha_

In [ ]:
l_alpha2 = [0.00001,0.00002,0.00003,0.00004,0.00005, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005,
           0.0006, 0.0007, 0.0008]

lasso_model=make_pipeline(RobustScaler(),
                         LassoCV(alphas=l_alpha2,max_iter=1e5,cv=kfolds)).fit(X2,y)
cv_rmse2(lasso_model).mean()

In [ ]:
lasso_model.steps[1][1].alpha_

In [ ]:
from sklearn.linear_model import ElasticNetCV

e_alphas = [0.00001,0.00005,0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.85, 0.9, 0.95, 0.99, 1]

elastic_cv=make_pipeline(RobustScaler(),
                        ElasticNetCV(max_iter=1e5,alphas=e_alphas,l1_ratio=e_l1ratio,cv=kfolds)).fit(X2,y)
cv_rmse2(elastic_cv).mean()

In [ ]:
elastic_cv.steps[1][1].alpha_

In [ ]:
elastic_cv.steps[1][1].l1_ratio_

In [ ]:
from lightgbm import LGBMRegressor

lgbm_model = LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [ ]:
cv_rmse(lgbm_model).mean()

In [ ]:
cv_rmse2(lgbm_model).mean()

In [ ]:
np.expm1(0.381)

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
xgb3 = XGBRegressor(learning_rate =0.01, n_estimators=3460, max_depth=3,
                     min_child_weight=0 ,gamma=0, subsample=0.7,
                     colsample_bytree=0.7,objective= 'reg:linear',
                     nthread=4,scale_pos_weight=1,seed=27, reg_alpha=0.00006)



In [ ]:
cv_rmse(xgb3).mean()

In [ ]:
cv_rmse2(xgb3).mean()

In [ ]:
xgb_fit=xgb3.fit(X,y)